Plot the locations of the profiles for the different regions. 

In [15]:
import os
import glob
import netCDF4
import logging
import numpy as np
import emodnetchemistry
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
import calendar
import warnings
import matplotlib.cbook
warnings.filterwarnings("ignore",category=matplotlib.cbook.mplDeprecation)
plt.rcParams.update({'font.size': 16})
plt.rc('figure', facecolor='w')
from importlib import reload
reload(emodnetchemistry)

<module 'emodnetchemistry' from '/home/ctroupin/Projects/EMODnet/Chemistry3/EMODnet-Chemistry/python/emodnetchemistry.py'>

In [2]:
logger = logging.getLogger("EMODnet-Chemistry-Data")
logger.setLevel(logging.DEBUG)
logging.info("Starting")

## Paths

In [3]:
datadir = "/data/EMODnet/Eutrophication/Split/"
figdir = "/home/ctroupin/dox/EMODnet-Chemistry/Histograms/"
if not os.path.exists(figdir):
    os.makedirs(figdir)
    
cbar_ax = fig.add_axes([0.15, 0.15, 0.7, 0.05])
    cb = plt.colorbar(pcm,  cax=cbar_ax, extend="both", orientation="horizontal")
    cb.set_label(units, rotation=0, ha="left")
    cb.set_ticks(np.arange(vmin, vmax + 0.0001, deltavar))
    if figname is not None:
        plt.savefig(figname, dpi=300, bbox_inches="tight")
    plt.close()

## Loop on the variables

In [14]:
for varname in varlist[3:4]:
    logger.info("Working on variable {}".format(varname))
    datafilelist = sorted(glob.glob(os.path.join(datadir, f"*{varname}*.nc")))
    nfiles = len(datafilelist)
    logger.info("Working on {} files".format(nfiles))
    datafilelist

    timeobs = np.array([])
    dateobs = np.array([])
    obsval = np.array([])
    for datafile in datafilelist:
        logger.info("Working on {}".format(datafile))
        with netCDF4.Dataset(datafile, "r") as nc:
            
            varnamenc = (list(nc.variables.keys())[-1])
            logger.info("Name in netCDF file: {}".format(varnamenc))
            timeunits = nc.variables["obstime"].units
            times = nc.variables["obstime"][:]
            values = nc.variables[varnamenc][:]
            dates = netCDF4.num2date(times, timeunits)
            timeobs = np.append(timeobs, times)
            dateobs = np.append(dateobs, dates)
            obsval = np.append(obsval, values)
        
    logger.info("Preparing arrays of years and months")
    years = [dd.year for dd in dateobs.data]
    months = [dd.month for dd in dateobs.data]
    
    logger.info("Creating histograms")
    emodnetchemistry.make_histo_values(obsval, varname, figdir)
    #emodnetchemistry.make_histo_month(months, varname, figdir)
    #emodnetchemistry.make_histo_year(years, varname, figdir)

INFO:EMODnet-Chemistry-Data:Working on variable phosphate
INFO:EMODnet-Chemistry-Data:Working on 9 files
INFO:EMODnet-Chemistry-Data:Working on /data/EMODnet/Eutrophication/Split/ArcticSea_Water_body_phosphate.nc
INFO:EMODnet-Chemistry-Data:Name in netCDF file: Water body phosphate
INFO:EMODnet-Chemistry-Data:Working on /data/EMODnet/Eutrophication/Split/Atlantic_TS_Water_body_phosphate.nc
INFO:EMODnet-Chemistry-Data:Name in netCDF file: Water body phosphate
INFO:EMODnet-Chemistry-Data:Working on /data/EMODnet/Eutrophication/Split/Atlantic_Water_body_phosphate.nc
INFO:EMODnet-Chemistry-Data:Name in netCDF file: Water body phosphate
INFO:EMODnet-Chemistry-Data:Working on /data/EMODnet/Eutrophication/Split/BalticSea_TS_Water_body_phosphate.nc
INFO:EMODnet-Chemistry-Data:Name in netCDF file: Water body phosphate
INFO:EMODnet-Chemistry-Data:Working on /data/EMODnet/Eutrophication/Split/BalticSea_Water_body_phosphate.nc
INFO:EMODnet-Chemistry-Data:Name in netCDF file: Water body phosphate
I

INFO:EMODnet-Chemistry-Data:Name in netCDF file: Water body dissolved inorganic nitrogen (DIN)
INFO:EMODnet-Chemistry-Data:Working on /data/EMODnet/Eutrophication/Split/BlackSea_Water_body_dissolved_inorganic_nitrogen_(DIN).nc
INFO:EMODnet-Chemistry-Data:Name in netCDF file: Water body dissolved inorganic nitrogen (DIN)
INFO:EMODnet-Chemistry-Data:Working on /data/EMODnet/Eutrophication/Split/MedSea2_TS_Water_body_dissolved_inorganic_nitrogen_(DIN).nc
INFO:EMODnet-Chemistry-Data:Name in netCDF file: Water body dissolved inorganic nitrogen (DIN)
INFO:EMODnet-Chemistry-Data:Working on /data/EMODnet/Eutrophication/Split/MedSea2_Water_body_dissolved_inorganic_nitrogen_(DIN).nc
INFO:EMODnet-Chemistry-Data:Name in netCDF file: Water body dissolved inorganic nitrogen (DIN)
INFO:EMODnet-Chemistry-Data:Working on /data/EMODnet/Eutrophication/Split/NorthSea_Water_body_dissolved_inorganic_nitrogen_(DIN).nc
INFO:EMODnet-Chemistry-Data:Name in netCDF file: Water body dissolved inorganic nitrogen (D